# MiniRocket

MiniRocket transforms input time series using a small, fixed set of convolutional kernels.  MiniRocket uses PPV pooling to compute a single feature for each of the resulting feature maps (i.e., the proportion of positive values). The transformed features are used to train a linear classifier.

Dempster A, Schmidt DF, Webb GI (2020) MiniRocket: A Very Fast (Almost) Deterministic Transform for Time Series Classification [arXiv:2012.08791](https://arxiv.org/abs/2012.08791)

## 1 Univariate Time Series

### 1.1 Imports

Import example data, MiniRocket, `RidgeClassifierCV` (scikit-learn), and NumPy.

**Note**: MiniRocket and MiniRocketMultivariate are compiled by Numba on import.  The compiled functions are cached, so this should only happen once (i.e., the first time you import MiniRocket or MiniRocketMultivariate).

In [1]:
# !pip install --upgrade numba

In [2]:
import numpy as np
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline

from sktime.datasets import load_arrow_head  # univariate dataset
from sktime.datasets import load_basic_motions  # multivariate dataset
from sktime.transformations.panel.rocket import MiniRocket, MiniRocketMultivariate

### 1.2 Load the Training Data

For more details on the data set, see the [univariate time series classification notebook](https://github.com/alan-turing-institute/sktime/blob/main/examples/02_classification_univariate.ipynb).

**Note**: Input time series must be *at least* of length 9.  Pad shorter time series using, e.g., `PaddingTransformer` (`sktime.transformers.panel.padder`).

In [3]:
X_train, y_train = load_arrow_head(split="train", return_X_y=True)

### 1.3 Initialise MiniRocket and Transform the Training Data

In [4]:
minirocket = MiniRocket()  # by default, MiniRocket uses ~10,000 kernels
minirocket.fit(X_train)
X_train_transform = minirocket.transform(X_train)

### 1.4 Fit a Classifier

We suggest using `RidgeClassifierCV` (scikit-learn) for smaller datasets (fewer than ~10,000 training examples), and using logistic regression trained using stochastic gradient descent for larger datasets.

**Note**: For larger datasets, this means integrating MiniRocket with stochastic gradient descent such that the transform is performed per minibatch, *not* simply substituting `RidgeClassifierCV` for, e.g., `LogisticRegression`.

In [5]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

### 1.5 Load and Transform the Test Data

In [6]:
X_test, y_test = load_arrow_head(split="test", return_X_y=True)
X_test_transform = minirocket.transform(X_test)

### 1.6 Classify the Test Data

In [7]:
classifier.score(X_test_transform, y_test)

0.8742857142857143

***

## 2 Multivariate Time Series

We can use the multivariate version of MiniRocket for multivariate time series input.

### 2.1 Imports

Import MiniRocketMultivariate.

**Note**: MiniRocketMultivariate compiles via Numba on import.

In [8]:
# (above)

### 2.2 Load the Training Data

**Note**: Input time series must be *at least* of length 9.  Pad shorter time series using, e.g., `PaddingTransformer` (`sktime.transformers.panel.padder`).

In [9]:
X_train, y_train = load_basic_motions(split="train", return_X_y=True)

### 2.3 Initialise MiniRocket and Transform the Training Data

In [10]:
minirocket_multi = MiniRocketMultivariate()
minirocket_multi.fit(X_train)
X_train_transform = minirocket_multi.transform(X_train)

### 2.4 Fit a Classifier

In [11]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

### 2.5 Load and Transform the Test Data

In [12]:
X_test, y_test = load_basic_motions(split="test", return_X_y=True)
X_test_transform = minirocket_multi.transform(X_test)

### 2.6 Classify the Test Data

In [13]:
classifier.score(X_test_transform, y_test)

1.0

***

## 3 Pipeline Example

We can use MiniRocket together with `RidgeClassifierCV` (or another classifier) in a pipeline.  We can then use the pipeline like a self-contained classifier, with a single call to `fit`, and without having to separately transform the data, etc.

### 3.1 Imports

In [14]:
# (above)

### 3.2 Initialise the Pipeline

In [15]:
minirocket_pipeline = make_pipeline(
    MiniRocket(), RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
)

### 3.3 Load and Fit the Training Data

**Note**: Input time series must be *at least* of length 9.  Pad shorter time series using, e.g., `PaddingTransformer` (`sktime.transformers.panel.padder`).

In [16]:
X_train, y_train = load_arrow_head(split="train", return_X_y=True)

# it is necessary to pass y_train to the pipeline
# y_train is not used for the transform, but it is used by the classifier
minirocket_pipeline.fit(X_train, y_train)

Pipeline(steps=[('minirocket', MiniRocket()),
                ('ridgeclassifiercv',
                 RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                                   normalize=True))])

### 3.4 Load and Classify the Test Data

In [17]:
X_test, y_test = load_arrow_head(split="test", return_X_y=True)

minirocket_pipeline.score(X_test, y_test)

0.8628571428571429